In [79]:
from typing import List
from utils import Reader, Logger, Parser
from utils import Repository, Mapper, Validator

## Implementations

In [80]:
class FileLogger(Logger):
        def __init__(self, filename):
            self.filename = filename

        def log(self, message):
             with open(self.filename, 'a') as f:
                    f.write(f'{message}\n')


In [81]:
class FileRepo(Repository):
        def __init__(self, filename):
            self.filename = filename

        def store_records(self, records):
             with open(self.filename, 'a') as f:
                for record in records:
                    f.write(f'{record}\n')


In [82]:
class Customer:
        def __init__(self, id, loc, product, amount):
            self.id = id
            self.loc = loc
            self.product = product
            self.amount = amount

        def __str__(self):
            return f'[{self.id}, {self.loc}, {self.product}, {self.amount}]'

In [83]:
class CustomerMapper(Mapper):
        def map(self, processed_record: List[str]):
            id = processed_record[0]
            location = processed_record[1]
            product = processed_record[2]
            amount = float(processed_record[3])
            customer = Customer(id, location,product, amount)
            return customer

In [84]:
class UnderscoreParser(Parser):
        def __init__(self, validator: Validator, mapper: Mapper):
            Parser.__init__(self, validator, mapper)

        def parse(self, record: str)-> List[str]:
            return record.split('_')

In [85]:
class FileReader(Reader):
        def __init__(self, filename):
            self.filename = filename
            
        def read_data(self):
            trade_records: List[str] = []
            with open(self.filename) as data_source:
                for trade_record in data_source: 
                    trade_records.append(trade_record.rstrip())
            return trade_records

Third party class

In [86]:
class FancyValidator:
    def approve(self, word: str):
        if len(word) == 10:
            if word.isalnum():
                return True
            else: 
                return False
        else:
            return False
            
    def isword(self, loc: str):
        if loc.isalpha():
                return True
        else: 
                return False
            
    def isnumber(self, number: str):
        if number.isdigit():
            return True
        else:
            return False

Adapter class

In [87]:
class Adapter(Validator):
    def __init__(self, logger: Logger, adaptee: FancyValidator):
        self.logger = logger
        self.adaptee = adaptee
        
        
    def validate(self, record: List[str], index: int) -> bool:
        if len(record) != 4: #check number of fields
            self.logger.log(f'Line {index} malformed. Only {len(record)} field(s) found.')
            return False
        if not self.adaptee.approve(record[0]): #check id
            self.logger.log(f'Id on line {index} malformed: {record[0]}')
            return False
        if not self.adaptee.isword(record[1]): #check location
            self.logger.log(f"Location on {index} is invalid: '{record[1]}'")
            return False
        if not self.adaptee.isword(record[2]): #check product
            self.logger.log(f"Product name on {index} is invalid: '{record[2]}'")
            return False
        if not self.adaptee.isnumber(record[3]):
            self.logger.log(f'Amount on line {index} not a valid int:{record[3]}')
            return False
        return True

In [88]:
class TradeProcessor:
    def __init__(self,reader: Reader, parser: Parser, repo: Repository) -> None:
        self.reader = reader
        self.parser = parser
        self.repo = repo

    def process_trades(self):
        lines = self.reader.read_data()
        trades = self.parser.parse_trades(lines)
        self.repo.store_records(trades)

In [89]:
mapper = CustomerMapper()
logger = FileLogger('run.log')

fancy_validator = FancyValidator()
validator = Adapter(logger, fancy_validator)
parser = UnderscoreParser(validator, mapper)

repo = FileRepo('store.txt')

reader = FileReader('Data2.txt')
tradeProcessor = TradeProcessor(reader, parser, repo)
tradeProcessor.process_trades()